# 

In [1]:
import selenium as sel
from selenium.webdriver.common.by import By
import pandas as pd
import time
import datetime
import base as bs
import ElTiempoFunciones as etf

import json
import re

import os as os
import csv

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
f = open('config.json')
empresas = json.load(f)

if not os.path.exists('../data/raw/eltiempo.csv'):
    os.makedirs('../data/raw/eltiempo.csv')
    
#for i in empresas['LaRepublica']:
    

In [3]:
#Empresa con la cual vamos a extraer los articulos
# TODO Tener una lista que itere por todos lo clientes
empresa = input("Digite la empresa a extraer: ")
paginas = input("la cantidad de paginas ")

In [4]:
#cerar driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()

In [5]:
def obtener_articulos(driver, url: str, titulares):
    """obtiene los ariculos de una pagina del tiempo dada la url.
        'https://www.eltiempo.com/buscar?q={empresa}'
        'https://www.eltiempo.com/buscar/{i}?q={empresa}'

    Args:
        driver (_type_): _description_
        url (str): _description_
        titulares (_type_): _description_
    """

    driver.get(url)
    driver.implicitly_wait(10)
    buscar = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]/div[2]/div[2]/div')
    articulos = buscar.find_elements(By.CLASS_NAME,"listing")

    for articulos in articulos:
            aux = articulos.find_element(By.XPATH, './/h3[contains(@class, "title-container")]')
            url = aux.find_element(By.XPATH,'.//a').get_attribute('href')
            # print(url)
            if not(bs.existedb(url, "eltiempo")):
                    titulo = articulos.find_element(By.CLASS_NAME,"title-container").text
                    # print(titulo)
                    resumen = articulos.find_element(By.CLASS_NAME,"epigraph-container").text
                    # print(resumen)
                    fechaPub = articulos.find_element(By.CLASS_NAME,"published-at").text
                    # print(fechaPub)
                    tema = articulos.find_element(By.CLASS_NAME,"category").text
                    # print(tema)
                    titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                    'Titulo':titulo,
                                    'Fecha Publicacion':fechaPub,
                                    'Tema':tema,
                                    'URL':url,
                                    'Resumen':resumen,
                                    'Empresa':empresa})


In [6]:
titulares=[]
get_url = driver.current_url
# hacer el WS del primer url
obtener_articulos(driver=driver, url=f'https://www.eltiempo.com/buscar?q={empresa}',titulares = titulares)

for i in range(2,int(paginas) + 1):
        obtener_articulos(driver=driver, url=f'https://www.eltiempo.com/buscar/{i}?q={empresa}',titulares = titulares)


### Extrae info de cada URL

In [7]:
# Inicializa los cookies del navegador 
driver.delete_all_cookies()

# Busca cada articulo y las almacena en la lista de titulares
for tit in titulares:
    # clear all cookies in scope of session
    driver.get(tit['URL'])
    driver.implicitly_wait(10) #Nueva metodología de wait


    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException)
    contenido = ''
    try :
        html = WebDriverWait(driver,10,ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.XPATH,'.//div[contains(@class,"modulos public-side")]')))
        parrafos = html.find_elements(By.XPATH,'.//p') 

    except:
        try:
            html = driver.find_element(By.XPATH,'.//div[contains(@class,"modulos public-side")]')
            parrafos = html.find_elements(By.XPATH,'.//p')
        except:
            contenido = 'SIN PARRAFOS'
        else:
            for i in parrafos:
                contenido += i.text

    else:
        try:
            for i in parrafos:
                contenido += i.text
        except:
            print('Error en el último for == el más dentro de "for tit in titulares:" externo')
    
    # agregar contenido al dict de titulares
    tit['Contenido'] = contenido
    driver.delete_all_cookies()
    
    #agregar lista de URLs de noticias relacionadas
    # tit['RelNewsUrls'] = bs.obtener_articulos_relacionados_eltiempo(driver)


### Para agregar la info en un archivo CSV

In [ ]:
for i in titulares:
    bs.guardar_articulo(pd.DataFrame(i))

In [8]:
columnas = ['Fecha Extraccion','Titulo', 'Fecha Publicacion','Resumen','URL','Empresa','Tema','Contenido']

# Para AGREGAR titulares a un archivo.
with open('../data/raw/eltiempo.csv', 'a', newline='', errors='ignore') as csv_file:
    dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
    dict_object.writerows(titulares)

In [9]:
# Imprimir los últimos datos de
titulares[-1]

# Cerrar navegador
driver.quit()